In [132]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
from datetime import time
import numpy as np

In [153]:
data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/MidWest_WFM_Stat_2025_05_16.xlsx')
data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/CS East WFM STAT 2025_05_14.xlsx')
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
               'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
               'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
data = data[~data .offered.isna()]
path_predict = 'workspace/Midwest_CS_predict_Improved'
path_test='workspace/Midwest_CS_test_Improved'
data_=data.copy()
path='workspace/Midwest_CS_Test_Improved_'
holiday_data = pd.read_excel(r'/workspace/ADP_SBS_Call_Center_Forecasting/Data/holiday.xlsx')
unique_times_Midwest = [ datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),datetime.time(18,0),datetime.time(18,30),datetime.time(19,0),datetime.time(19,30),datetime.time(20,0),datetime.time(20,30)
]
unique_times= [ datetime.time(7, 0), datetime.time(7, 30), datetime.time(8, 0)
                ,datetime.time(8, 30),datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),
                 datetime.time(18, 0), datetime.time(18, 30), datetime.time(19, 0)
]
month_list=['2025-02-15']
#change

    

for month_number in month_list:
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    matrix = []
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    # 获取所有唯一的日期
    unique_dates = data['date'].unique()
    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    holiday_datetime = holiday_data.date.to_numpy()
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length
    add_column = ['actans','actabn','absActHt','absActSa']
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(),end=unique_dates.max()+pd.offsets.BDay(start_time), freq='B') # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        # df=time_data.to_frame(name='offer')
        # df_last_year = df.copy()
        # df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
        # df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'})
        # df_last_two_year = df.copy()
        # df_last_two_year.index = df_last_two_year.index + pd.DateOffset(years=2)
        # df_last_two_year = df_last_two_year.rename(columns={'offer': 'offered_last_two_year'})        
        # # Step 3: Join on index
        # result = df.join(df_last_year, how='left').join(df_last_two_year,how='left')
        # year_data=result[['offered_last_year','offered_last_two_year']]
        # 将当前时间点、前半小时和后半小时的数据拼接
        #combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2], axis=1)
        #combined_data = combined_data.merge(date_column, left_index=True, right_index=True, how='left')
        fiscal_start_month = 7  # July
        # Fiscal Year
        combined_data['fiscalYear'] = combined_data.index.map(
            lambda d: f"FY {d.year + 1}" if d.month >= fiscal_start_month else f"FY {d.year}"
        )
        # Fiscal Month Index (1 to 12 from fiscal start)
        combined_data['fiscalMonth'] = combined_data.index.map(
            lambda d: d.strftime('%B')
        )
        fiscal_month_index = combined_data.index.month - fiscal_start_month + 1
        combined_data['fiscalMonthIndex'] = fiscal_month_index.where(fiscal_month_index > 0, fiscal_month_index + 12)
        # Fiscal Quarter
        combined_data['ficalQuarter'] = combined_data['fiscalMonthIndex'].map(lambda m: f"Q{((m-1)//3)+1}")
        # Fiscal Week
        combined_data['fiscalWeek'] = combined_data.index.isocalendar().week
        # Day of Week
        combined_data['DOW'] = combined_data.index.strftime('%A')
        combined_data.drop(columns='fiscalMonthIndex', inplace=True)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in combined_data.index]).reshape(-1, 1)
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
            
    

    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['fiscalYear', 'fiscalMonth','ficalQuarter', 'fiscalWeek', 'DOW']).sort_values('datetime').reset_index(drop = True)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    read_data = matrix.iloc[:]
    # Quarter: 'Q1' → 1, ..., 'Q4' → 4
    quarter_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4}
    read_data['ficalQuarter'] = read_data['ficalQuarter'].map(quarter_map)
    year_map={'FY 2021':1 ,'FY 2022':2,'FY 2023':3,'FY 2024':4, 'FY 2025':5}
    read_data['fiscalYear']=read_data['fiscalYear'].map(year_map)
    # Month: 'January' → 1, ..., 'December' → 12
    month_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4,
        'May': 5, 'June': 6, 'July': 7, 'August': 8,
        'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    read_data['fiscalMonth'] = read_data['fiscalMonth'].map(month_map)
    # Day of week: 'Monday' → 0, ..., 'Sunday' → 6
    dow_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
        'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6
    }
    read_data['DOW'] = read_data['DOW'].map(dow_map)
    read_data['fiscalWeek']=read_data['fiscalWeek'].apply(lambda x:pd.to_numeric(x))

    # Step 2: Apply cyclical encoding
    read_data['ficalQuarter_sin'] = np.sin(2 * np.pi * read_data['ficalQuarter'] / 4)
    read_data['ficalQuarter_cos'] = np.cos(2 * np.pi * read_data['ficalQuarter'] / 4)
    read_data['fiscalMonth_sin'] = np.sin(2 * np.pi * read_data['fiscalMonth'] / 12)
    read_data['fiscalMonth_cos'] = np.cos(2 * np.pi * read_data['fiscalMonth'] / 12)
    read_data['fiscalWeek_sin'] = np.sin(2 * np.pi * read_data['fiscalWeek'] / 52)
    read_data['fiscalWeek_cos'] = np.cos(2 * np.pi * read_data['fiscalWeek'] / 52)
    read_data['DOW_sin'] = np.sin(2 * np.pi * read_data['DOW'] / 7)
    read_data['DOW_cos'] = np.cos(2 * np.pi * read_data['DOW'] / 7) 
    # Hour: 0 to 23
    read_data['hour_sin'] = np.sin(2 * np.pi * read_data['hour'] / 24)
    read_data['hour_cos'] = np.cos(2 * np.pi * read_data['hour'] / 24)
    # Minute: 0 to 59
    read_data['minute_sin'] = np.sin(2 * np.pi * read_data['minute'] / 60)
    read_data['minute_cos'] = np.cos(2 * np.pi * read_data['minute'] / 60)
    # Step 3: Drop original raw categorical time features
    read_data.drop(['ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW'], axis=1, inplace=True)
    fill_na = read_data.iloc[:,2:].apply(lambda x:pd.to_numeric(x))
    fill_na = fill_na.interpolate(method='linear')
    fill_na.fillna(method='ffill', inplace=True)
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na_predict=fill_na.copy()

/tmp/ipykernel_611528/454909371.py:44: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_611528/454909371.py:194: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fill_na.fillna(method='ffill', inplace=True)


In [162]:
data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/MidWest_WFM_Stat_2025_05_16.xlsx')
data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/CS East WFM STAT 2025_05_14.xlsx')
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
               'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
               'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
data = data[~data .offered.isna()]
path_predict = 'workspace/Midwest_CS_predict_Improved'
path_test='workspace/Midwest_CS_test_Improved'
data_=data.copy()
path='workspace/Midwest_CS_Test_Improved_'
holiday_data = pd.read_excel(r'/workspace/ADP_SBS_Call_Center_Forecasting/Data/holiday.xlsx')
unique_times_Midwest = [ datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),datetime.time(18,0),datetime.time(18,30),datetime.time(19,0),datetime.time(19,30),datetime.time(20,0),datetime.time(20,30)
]
unique_times= [ datetime.time(7, 0), datetime.time(7, 30), datetime.time(8, 0)
                ,datetime.time(8, 30),datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),
                 datetime.time(18, 0), datetime.time(18, 30), datetime.time(19, 0)
]
month_list=['2025-02-15']
#change
for month_number in month_list:
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    matrix = []
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    unique_dates = data['date'].unique()
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    # date_column=data[['date',
    #    'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']]
    # date_column=date_column.set_index('date')
    # date_column = date_column[~date_column.index.duplicated(keep='first')]
    # date_column=date_column.sort_values('date')
    # 获取所有唯一的日期

    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    
    holiday_datetime = holiday_data.date.to_numpy()
    
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length

    
    add_column = ['actans','actabn','absActHt','absActSa']
    
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        # df=time_data.to_frame(name='offer')
        # df_last_year = df.copy()
        # df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
        # df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'})
        # df_last_two_year = df.copy()
        # df_last_two_year.index = df_last_two_year.index + pd.DateOffset(years=2)
        # df_last_two_year = df_last_two_year.rename(columns={'offer': 'offered_last_two_year'})        
        # Step 3: Join on index
        # result = df.join(df_last_year, how='left').join(df_last_two_year,how='left')
        # year_data=result[['offered_last_year','offered_last_two_year']]
        #year_data = time_data.shift(251)
        # 将当前时间点、前半小时和后半小时的数据拼接
        # combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2], axis=1)
        # combined_data = combined_data.merge(date_column, left_index=True, right_index=True, how='left')
        # 创建日期+时间列
        fiscal_start_month = 7  # July
        # Fiscal Year
        combined_data['fiscalYear'] = combined_data.index.map(
            lambda d: f"FY {d.year + 1}" if d.month >= fiscal_start_month else f"FY {d.year}"
        )
        # Fiscal Month Index (1 to 12 from fiscal start)
        combined_data['fiscalMonth'] = combined_data.index.map(
            lambda d: d.strftime('%B')
        )
        fiscal_month_index = combined_data.index.month - fiscal_start_month + 1
        combined_data['fiscalMonthIndex'] = fiscal_month_index.where(fiscal_month_index > 0, fiscal_month_index + 12)
        # Fiscal Quarter
        combined_data['ficalQuarter'] = combined_data['fiscalMonthIndex'].map(lambda m: f"Q{((m-1)//3)+1}")
        # Fiscal Week
        combined_data['fiscalWeek'] = combined_data.index.isocalendar().week
        # Day of Week
        combined_data['DOW'] = combined_data.index.strftime('%A')
        combined_data.drop(columns='fiscalMonthIndex', inplace=True)
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in combined_data.index]).reshape(-1, 1)
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['fiscalYear', 'fiscalMonth', 'ficalQuarter', 'fiscalWeek', 'DOW']).sort_values('datetime').reset_index(drop = True)
    
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    
    #read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    # Quarter: 'Q1' → 1, ..., 'Q4' → 4
    quarter_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4}
    read_data['ficalQuarter'] = read_data['ficalQuarter'].map(quarter_map)
    year_map={'FY 2021':1 ,'FY 2022':2,'FY 2023':3,'FY 2024':4, 'FY 2025':5}
    read_data['fiscalYear']=read_data['fiscalYear'].map(year_map)
    # Month: 'January' → 1, ..., 'December' → 12
    month_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4,
        'May': 5, 'June': 6, 'July': 7, 'August': 8,
        'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    read_data['fiscalMonth'] = read_data['fiscalMonth'].map(month_map)
    
    # Day of week: 'Monday' → 0, ..., 'Sunday' → 6
    dow_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
        'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6
    }
    read_data['DOW'] = read_data['DOW'].map(dow_map)
    read_data['fiscalWeek']=read_data['fiscalWeek'].apply(lambda x:pd.to_numeric(x))
    # Step 2: Apply cyclical encoding
    
    read_data['ficalQuarter_sin'] = np.sin(2 * np.pi * read_data['ficalQuarter'] / 4)
    read_data['ficalQuarter_cos'] = np.cos(2 * np.pi * read_data['ficalQuarter'] / 4)
    
    read_data['fiscalMonth_sin'] = np.sin(2 * np.pi * read_data['fiscalMonth'] / 12)
    read_data['fiscalMonth_cos'] = np.cos(2 * np.pi * read_data['fiscalMonth'] / 12)
    
    read_data['fiscalWeek_sin'] = np.sin(2 * np.pi * read_data['fiscalWeek'] / 52)
    read_data['fiscalWeek_cos'] = np.cos(2 * np.pi * read_data['fiscalWeek'] / 52)
    
    read_data['DOW_sin'] = np.sin(2 * np.pi * read_data['DOW'] / 7)
    read_data['DOW_cos'] = np.cos(2 * np.pi * read_data['DOW'] / 7)
    # Hour: 0 to 23
    read_data['hour_sin'] = np.sin(2 * np.pi * read_data['hour'] / 24)
    read_data['hour_cos'] = np.cos(2 * np.pi * read_data['hour'] / 24)
    # Minute: 0 to 59
    read_data['minute_sin'] = np.sin(2 * np.pi * read_data['minute'] / 60)
    read_data['minute_cos'] = np.cos(2 * np.pi * read_data['minute'] / 60)
    # Step 3: Drop original raw categorical time features
    read_data.drop(['ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW'], axis=1, inplace=True)
    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

    fill_na = fill_na.interpolate(method='linear')
    #fill_na.fillna(method='ffill', inplace=True)
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)

/tmp/ipykernel_611528/2719531350.py:42: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')


In [164]:
def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

# set up the interval need to forecast here
unique_times = [ datetime.time(7, 0), datetime.time(7, 30), datetime.time(8, 0)
                ,datetime.time(8, 30),datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),
                 datetime.time(18, 0), datetime.time(18, 30), datetime.time(19, 0)
]

for month_number in ['2025-02-15']:
    #ETL Process
    data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/CS East WFM STAT 2025_05_14.xlsx')
    pd.set_option('display.max_rows', 100)
    data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
           'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
           'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
    
    data = data[~data .offered.isna()]
    
    data = data[data .offered != 0]
    # Month one and half need to set up 
    data_=data
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    holiday_data = pd.read_excel(r'/workspace/ADP_SBS_Call_Center_Forecasting/Data/holiday.xlsx')
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    
    # 获取所有唯一的时间点（如每天的09:00, 09:30等）
    #unique_times = data['time'].unique()
    
    # 获取所有唯一的日期
    unique_dates = data['date'].unique()
    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    
    holiday_datetime = holiday_data.date.to_numpy()
    
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length

    
    add_column = ['actans','actabn','absActHt','absActSa']
    
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    matrix = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    
    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))
    fill_na = fill_na.interpolate(method='linear') 
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

/tmp/ipykernel_611528/195476643.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
/tmp/ipykernel_611528/195476643.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.datetime.dt.date
/tmp/ipykernel_611528/195476643.py:61: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_611528/195476643.py:61: SettingW

In [165]:
fill_na.head(10)

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute,datetime,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2022,year_2023,year_2024,year_2025,week_of_year,weekday,week_of_year_1,week_of_year_2,week_of_year_3,week_of_year_4,week_of_year_5,week_of_year_6,week_of_year_7,week_of_year_8,week_of_year_9,week_of_year_10,week_of_year_11,week_of_year_12,week_of_year_13,week_of_year_14,week_of_year_15,week_of_year_16,week_of_year_17,week_of_year_18,week_of_year_19,week_of_year_20,week_of_year_21,week_of_year_22,week_of_year_23,week_of_year_24,week_of_year_25,week_of_year_26,week_of_year_27,week_of_year_28,week_of_year_29,week_of_year_30,week_of_year_31,week_of_year_32,week_of_year_33,week_of_year_34,week_of_year_35,week_of_year_36,week_of_year_37,week_of_year_38,week_of_year_39,week_of_year_40,week_of_year_41,week_of_year_42,week_of_year_43,week_of_year_44,week_of_year_45,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,11.0,16.0,1.0,18145.333333,4528.666667,14.0,1.0,13812.333333,399.000000,12.0,0.0,7126.200000,187.500000,7.0,0.0,10729.833333,574.000000,1

In [163]:
fill_na.head(10)

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,fiscalYear,hour,minute,ficalQuarter_sin,ficalQuarter_cos,fiscalMonth_sin,fiscalMonth_cos,fiscalWeek_sin,fiscalWeek_cos,DOW_sin,DOW_cos,hour_sin,hour_cos,minute_sin,minute_cos,datetime,date
0,14.0,5.0,0.0,7869.333333,169.333333,14.0,0.0,20975.355556,363.557143,15.0,1.0,22427.000000,390.085714,7.0,0.0,11785.600000,102.857143,12.0,1.0,12750.500000,244.000000,16.0,0.0,17596.349206,345.666667,13.0,0.0,13305.444444,270.044444,17.0,0.0,17171.400000,596.571429,20.0,1.0,25721.696970,1379.636364,23.0,0.0,30302.884615,493.589744,18.0,0.0,19743.333333,490.441176,11.0,0.0,14264.250000,385.846154,19.0,0.0,22775.088889,839.402597,2.0,0.0,808.500000,69.000000,19.0,0.0,21307.414286,386.785714,9.0,0.0,13209.000000,254.333333,18.0,0.0,28271.948052,408.000000,21.0,0.0,20502.900000,731.858824,26.0,1.0,27202.800000,3394.425339,8.0,0.0,9819.000000,141.750000,20.0,1.0,31067.458333,551.483333,24.0,0.0,22846.153846,464.000000,16.0,0.0,19141.333333,1513.555556,20.0,1.0,22566.250000,3984.418301,16.0,0.0,32312.000000,520.000000,12.0,0.0,10782.171429,366.476190,17.0,0.0,12187.142857,370.416667,6.0,0.0,12736.000000,1719.000000,22.0,0.0,29554.233333,504.761538,23.0,0.0,26697.066667,602.307692,15.0,0.0,19810.714286,397.000000,9.0,0.0,12781.200000,255.000000,6.0,0.0,61

In [146]:
lstm_data = fill_na
lstm_data
    
for i in lstm_data.columns:
    try:
        fill_na_predict[i]
    except:
        fill_na_predict[i] = 0

In [148]:
lstm_data.head(10)

,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,fiscalYear,hour,minute,ficalQuarter_sin,ficalQuarter_cos,fiscalMonth_sin,fiscalMonth_cos,fiscalWeek_sin,fiscalWeek_cos,DOW_sin,DOW_cos,hour_sin,hour_cos,minute_sin,minute_cos,datetime,date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,7,0,1.224647e-16,-1.0,-2.449294e-16,1.0,-0.120537,0.992709,0.7

In [20]:
holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date
data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')
date_column=data[['date',
   'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']]
date_column=date_column.set_index('date')
date_column = date_column[~date_column.index.duplicated(keep='first')]
date_column=date_column.sort_values('date')
# 获取所有唯一的日期

data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

holiday_datetime = holiday_data.date.to_numpy()

start_time = 35+23 # 58 how many days in advance
end_time = start_time+36 #36 is the train length


add_column = ['actans','actabn','absActHt','absActSa']

add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
for month_number in month_list:
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    unique_dates = data['date'].unique()
    matrix = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        df=time_data.to_frame(name='offer')
        df_last_year = df.copy()
        df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
        df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'})
        df_last_two_year = df.copy()
        df_last_two_year.index = df_last_two_year.index + pd.DateOffset(years=2)
        df_last_two_year = df_last_two_year.rename(columns={'offer': 'offered_last_two_year'})        
        # Step 3: Join on index
        result = df.join(df_last_year, how='left').join(df_last_two_year,how='left')
        year_data=result[['offered_last_year','offered_last_two_year']]
        #year_data = time_data.shift(251)
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        combined_data = combined_data.merge(date_column, left_index=True, right_index=True, how='left')

        # 创建日期+时间列
        #datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in combined_data.index]).reshape(-1, 1)
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data_1','years_data_2','fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']).sort_values('datetime').reset_index(drop = True)
    
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    
    #read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    # Quarter: 'Q1' → 1, ..., 'Q4' → 4
    quarter_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4}
    read_data['ficalQuarter'] = read_data['ficalQuarter'].map(quarter_map)
    year_map={'FY 2021':1 ,'FY 2022':2,'FY 2023':3,'FY 2024':4, 'FY 2025':5}
    read_data['fiscalYear']=read_data['fiscalYear'].map(year_map)
    # Month: 'January' → 1, ..., 'December' → 12
    month_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4,
        'May': 5, 'June': 6, 'July': 7, 'August': 8,
        'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    read_data['fiscalMonth'] = read_data['fiscalMonth'].map(month_map)
    
    # Day of week: 'Monday' → 0, ..., 'Sunday' → 6
    dow_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
        'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6
    }
    read_data['DOW'] = read_data['DOW'].map(dow_map)
    read_data['fiscalWeek']=read_data['fiscalWeek'].apply(lambda x:pd.to_numeric(x))
    # Step 2: Apply cyclical encoding
    
    read_data['ficalQuarter_sin'] = np.sin(2 * np.pi * read_data['ficalQuarter'] / 4)
    read_data['ficalQuarter_cos'] = np.cos(2 * np.pi * read_data['ficalQuarter'] / 4)
    
    read_data['fiscalMonth_sin'] = np.sin(2 * np.pi * read_data['fiscalMonth'] / 12)
    read_data['fiscalMonth_cos'] = np.cos(2 * np.pi * read_data['fiscalMonth'] / 12)
    
    read_data['fiscalWeek_sin'] = np.sin(2 * np.pi * read_data['fiscalWeek'] / 52)
    read_data['fiscalWeek_cos'] = np.cos(2 * np.pi * read_data['fiscalWeek'] / 52)
    
    read_data['DOW_sin'] = np.sin(2 * np.pi * read_data['DOW'] / 7)
    read_data['DOW_cos'] = np.cos(2 * np.pi * read_data['DOW'] / 7)
    # Hour: 0 to 23
    read_data['hour_sin'] = np.sin(2 * np.pi * read_data['hour'] / 24)
    read_data['hour_cos'] = np.cos(2 * np.pi * read_data['hour'] / 24)
    # Minute: 0 to 59
    read_data['minute_sin'] = np.sin(2 * np.pi * read_data['minute'] / 60)
    read_data['minute_cos'] = np.cos(2 * np.pi * read_data['minute'] / 60)
    # Step 3: Drop original raw categorical time features
    read_data.drop(['ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW'], axis=1, inplace=True)
    fill_na_ = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

    fill_na_ = fill_na.interpolate(method='linear')
    
    fill_na_['datetime'] = read_data.datetime
    fill_na_['date'] = fill_na_.datetime.dt.date
    fill_na_ = fill_na_.dropna().reset_index(drop = True)

/tmp/ipykernel_611528/714015081.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_611528/714015081.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)


KeyError: '~is_holiday'

In [ ]:
def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")
    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape
def calculate_mape(actual, predicted):
    actual = np.array(actual)
    predicted = np.array(predicted)
    mask = actual != 0
    if not np.any(mask):
        return np.nan  # Or return 0 or raise a different warning

    mape = np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100
    return mape

def ETL(month_list,data,holiday_data,unique_times):
    '''
    month_list is the list you want to forecast one and half month in advanced, for example
    , if you want to forecast May, 2025, please input ['2025-03-15'], also you can include multiple value in the list
    , like ['2024-11-15','2024-12-15','2025-01-15','2025-02-15','2025-03-15'];
    data is the source data in dataframe;
    holiday_data is the holiday data in dataframe
    full_dates are all the Business days you want to feed into Model for training;
    unique_times are the intervals need to forecast for every day;
    '''

    for month_number in month_list:
        data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
        matrix = []
        holiday_data['~is_holiday'] = 0
        Holiday_name = ['Christmas Day', 'Columbus Day',
               'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
               'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
               'Veterans Day']
        holiday_data['date'] = holiday_data.Date.dt.date
        data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
        data['date'] = data.datetime.dt.date
        unique_dates = data['date'].unique()
        data.datetime = data.datetime.dt.floor('T')
        data['datetime'] = data['datetime'].apply(
            lambda x: x.ceil('30T') if x.minute == 29 else x
        )
        data['datetime'] = data['datetime'].apply(
            lambda x: x.ceil('H') if x.minute == 59 else x
        )
        data['time'] = data.datetime.dt.time
        data = data.sort_values('datetime').reset_index(drop = True)
        data = data.sort_values(by='datetime')
        # date_column=data[['date',
        #    'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']]
        # date_column=date_column.set_index('date')
        # date_column = date_column[~date_column.index.duplicated(keep='first')]
        # date_column=date_column.sort_values('date')
        # 获取所有唯一的日期
    
        data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
        data = data[data['~is_holiday'] == 1]
        
        holiday_datetime = holiday_data.date.to_numpy()
        
        start_time = 35+23 # 58 how many days in advance
        end_time = start_time+36 #36 is the train length
    
        
        add_column = ['actans','actabn','absActHt','absActSa']
        
        add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                        [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                        [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                        [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
        
        # 遍历每个时间点
        for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
            # 过滤出当前时间点的数据
            time_data = data[data['time'] == time].set_index('date')['offered']    
            full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
            full_dates = full_dates.difference(holiday_datetime)
            time_data = time_data.groupby(time_data.index).sum()
            time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
            
            
            add_data = data[data['time'] == time].set_index('date')[add_column]
            add_data = add_data.groupby(add_data.index).sum()
            add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
            
            
            time_data1 = data[data['time'] == time1].set_index('date')['offered']    
            time_data1 = time_data1.groupby(time_data1.index).sum()
            time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
            
            time_data2 = data[data['time'] == time2].set_index('date')['offered']    
            time_data2 = time_data2.groupby(time_data2.index).sum()
            time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
            df=time_data.to_frame(name='offer')
            df_last_year = df.copy()
            df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
            df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'})
            df_last_two_year = df.copy()
            df_last_two_year.index = df_last_two_year.index + pd.DateOffset(years=2)
            df_last_two_year = df_last_two_year.rename(columns={'offer': 'offered_last_two_year'})        
            # Step 3: Join on index
            result = df.join(df_last_year, how='left').join(df_last_two_year,how='left')
            year_data=result[['offered_last_year','offered_last_two_year']]
            #year_data = time_data.shift(251)
            # 将当前时间点、前半小时和后半小时的数据拼接
            combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
            
            # 创建日期+时间列
            datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
            
            # 将日期+时间列添加到数据中
            combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
            
            # 将结果存入矩阵
            matrix.append(combined_data_with_datetime)
        
        max_rows = max(arr.shape[0] for arr in matrix)
        
        # 将每个时间点的数据填充到最大行数
        for i in range(len(matrix)):
            if matrix[i].shape[0] < max_rows:
                padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
                matrix[i] = np.vstack([matrix[i], padding])
        
        # 将矩阵堆叠成一个大的二维数组
        matrix = np.vstack(matrix)
        
        matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                              + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                              [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                               ['years_data_1','years_data_2']).sort_values('datetime').reset_index(drop = True)
        
        matrix['year'] = matrix.datetime.map(lambda x:x.year)
        matrix['month'] = matrix.datetime.map(lambda x:x.month)
        matrix['day'] = matrix.datetime.map(lambda x:x.day)
        matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
        matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
        
        matrix = matrix.sort_values('datetime').reset_index(drop = True)
        
        read_data = matrix.iloc[:]
        read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
        
        fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))
        fill_na = fill_na.interpolate(method='linear')
        
        fill_na['datetime'] = read_data.datetime
        fill_na['date'] = fill_na.datetime.dt.date
        fill_na = fill_na.dropna().reset_index(drop = True)
        
        fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
        fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
        fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
        fill_na['weekday'] = fill_na['datetime'].dt.day_name()
        fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
        fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)
    return fill_na

def ETL_(month_list,data,holiday_data,unique_times):
    '''
    month_list is the list you want to forecast one and half month in advanced, for example
    , if you want to forecast May, 2025, please input ['2025-03-15'], also you can include multiple value in the list
    , like ['2024-11-15','2024-12-15','2025-01-15','2025-02-15','2025-03-15'];
    data is the source data in dataframe;
    holiday_data is the holiday data in dataframe
    full_dates are all the Business days you want to feed into Model for training;
    unique_times are the intervals need to forecast for every day;
    '''

    for month_number in month_list:
        data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
        matrix = []
        holiday_data['~is_holiday'] = 0
        Holiday_name = ['Christmas Day', 'Columbus Day',
               'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
               'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
               'Veterans Day']
        holiday_data['date'] = holiday_data.Date.dt.date
        data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
        data['date'] = data.datetime.dt.date
        data.datetime = data.datetime.dt.floor('T')
        data['datetime'] = data['datetime'].apply(
            lambda x: x.ceil('30T') if x.minute == 29 else x
        )
        data['datetime'] = data['datetime'].apply(
            lambda x: x.ceil('H') if x.minute == 59 else x
        )
        data['time'] = data.datetime.dt.time
        data = data.sort_values('datetime').reset_index(drop = True)
        data = data.sort_values(by='datetime')
        # 获取所有唯一的日期
        unique_dates = data['date'].unique()
        data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
        data = data[data['~is_holiday'] == 1]
        holiday_datetime = holiday_data.date.to_numpy()
        start_time = 35+23 # 58 how many days in advance
        end_time = start_time+36 #36 is the train length
        add_column = ['actans','actabn','absActHt','absActSa']
        add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                        [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                        [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                        [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
        
        # 遍历每个时间点
        for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
            # 过滤出当前时间点的数据
            time_data = data[data['time'] == time].set_index('date')['offered']    
            full_dates = pd.date_range(start=unique_dates.min(),end=unique_dates.max()+pd.offsets.BDay(start_time), freq='B') # 仅工作日
            full_dates = full_dates.difference(holiday_datetime)
            time_data = time_data.groupby(time_data.index).sum()
            time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
            
            
            add_data = data[data['time'] == time].set_index('date')[add_column]
            add_data = add_data.groupby(add_data.index).sum()
            add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
            
            
            time_data1 = data[data['time'] == time1].set_index('date')['offered']    
            time_data1 = time_data1.groupby(time_data1.index).sum()
            time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
            
            time_data2 = data[data['time'] == time2].set_index('date')['offered']    
            time_data2 = time_data2.groupby(time_data2.index).sum()
            time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
            shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
            df=time_data.to_frame(name='offer')
            df_last_year = df.copy()
            df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
            df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'})
            df_last_two_year = df.copy()
            df_last_two_year.index = df_last_two_year.index + pd.DateOffset(years=2)
            df_last_two_year = df_last_two_year.rename(columns={'offer': 'offered_last_two_year'})        
            # Step 3: Join on index
            result = df.join(df_last_year, how='left').join(df_last_two_year,how='left')
            year_data=result[['offered_last_year','offered_last_two_year']]
        
            # 将当前时间点、前半小时和后半小时的数据拼接
            combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
            
            # 创建日期+时间列
            datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
            combined_data = combined_data.copy()
            combined_data['datetime'] = [pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in combined_data.index]
            combined_data_with_datetime = combined_data.reset_index(drop=True).to_numpy()[-start_time:]
            # 将日期+时间列添加到数据中           
            
            # 将结果存入矩阵
            matrix_predict.append(combined_data_with_datetime)
        
        max_rows = max(arr.shape[0] for arr in matrix_predict)
        # 将每个时间点的数据填充到最大行数
        for i in range(len(matrix_predict)):
            if matrix_predict[i].shape[0] < max_rows:
                padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
                matrix_predict[i] = np.vstack([matrix_predict[i], padding])
        
        # 将矩阵堆叠成一个大的二维数组
        matrix_predict = np.vstack(matrix_predict)
        
        matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                              + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                               ['years_data_1','years_data_2']).sort_values('datetime').reset_index(drop = True)
        
        matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
        matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
        matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
        matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
        matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)
        
        matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)
        
        read_data_predict = matrix_predict.iloc[:]
        
        
        
        
        fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))
        
        fill_na_predict = fill_na_predict.interpolate(method='linear')
        fill_na_predict = fill_na_predict.fillna(1)
        
        fill_na_predict['datetime'] = read_data_predict.datetime
        fill_na_predict['date'] = fill_na_predict.datetime.dt.date
        
        fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
        fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
        fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
        fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
        fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
        fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
        
        fill_na_predict

    return fill_na

def Train_Model(train_dataset,test_dataset,unique_times):
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length
    all_feature = pd.DataFrame()
    all_predict=pd.DataFrame()
    for (hour,minute) in [(i.hour,i.minute) for i in unique_times]:
        now_lstm_data = train_dataset[(train_dataset.hour == hour)&(train_dataset.minute == minute)]
        now_fill_na_predict = test_dataset[(test_dataset.hour == hour)&(test_dataset.minute == minute)]
        now_time_list = now_lstm_data.datetime.to_numpy()

        now_lstm_data = now_lstm_data.reset_index(drop = True)
        # 归一化数据
        
        # Separate the parts
        to_scale_df = now_lstm_data[cols_offered]
        keep_df = now_lstm_data[cal_cols]
        scaler_y = MinMaxScaler(feature_range=(0, 1))
        scaler_x = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler_y.fit_transform(now_lstm_data[['offered']])
        scaled_array = scaler_x.fit_transform(to_scale_df)
        scaled_df = pd.DataFrame(scaled_array, columns=cols_offered)
        # Concatenate back together
        X_combined = pd.concat([keep_df.reset_index(drop=True), scaled_df.reset_index(drop=True)], axis=1)
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =X_combined [:-start_time] , scaled_data[:-start_time]
        X_test_1, y_test =X_combined[-start_time:]  , scaled_data[-start_time:]
        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        X_train_1 = X_train_1.astype(np.float32)
        y_train = y_train.astype(np.float32)
        X_test_1 = X_test_1.astype(np.float32)
        y_test = y_test.astype(np.float32)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(Dense(25))
        model_1.add(LSTM(20, return_sequences=False))
        model_1.add(Dense(1))
    
        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler_y.inverse_transform(train_predict_1)
        True_data_1 = scaler_y.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler_y.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-start_time:]
        now_lstm_data = now_lstm_data[['offered'] +cols_].reset_index(drop = True)        
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_2 = X_combined[:-start_time]
        X_test_2 = X_combined[-start_time:]
        y_train = scaled_data[:-start_time]
        y_test = scaled_data[-start_time:]
        
        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        X_train_2 = X_train_2.astype(np.float32)
        y_train = y_train.astype(np.float32)
        X_test_2 = X_test_2.astype(np.float32)
        y_test = y_test.astype(np.float32)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))
    
        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler_y.inverse_transform(train_predict_2)
        True_data_2 = scaler_y.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler_y.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-start_time:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
            all_predict = pd.concat([now_predict_1,all_predict])
            future_X = now_fill_na_predict[cols_].to_numpy()
            future_data = scaler_y.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        else:
            all_predict = pd.concat([now_predict_2,all_predict])
            future_X = now_fill_na_predict[cols_].to_numpy()
            future_data = scaler_y.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        #print('Interval Hour:{}, Minute:{} has been trained'.format(hour,minute)
    all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
    #all_feature.to_excel(r'return_month_{}.xlsx'.format(month_number))
    all_predict['diff'] = (all_predict.predict - all_predict.offered)
    all_predict['hour'] = all_predict.datetime.dt.hour
    pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    calculate_mape(all_predict.offered,all_predict.predict)
    
    time_ = []
    for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        time_.append(now_fill_na_predict.datetime.to_numpy())
    all_feature['datetime'] =  np.hstack(time_)
    all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
    data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
    all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
    all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
    all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
    all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    month_number='2025-02-15'
    all_predict.to_excel(f'{path_test}_{month_number}_{timestamp}.xlsx')
    all_feature.to_excel(f'{path_predict}_{month_number}_{timestamp}.xlsx')
    return all_predict, all_feature

data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/MidWest_WFM_Stat_2025_05_16.xlsx')
data = pd.read_excel('/workspace/ADP_SBS_Call_Center_Forecasting/Data/CS East WFM STAT 2025_05_14.xlsx')
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
               'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
               'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
data = data[~data .offered.isna()]
path_predict = 'workspace/Midwest_CS_predict_Improved'
path_test='workspace/Midwest_CS_test_Improved'
data_=data.copy()
path='workspace/Midwest_CS_Test_Improved_'
holiday_data = pd.read_excel(r'/workspace/ADP_SBS_Call_Center_Forecasting/Data/holiday.xlsx')
unique_times_Midwest = [ datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),datetime.time(18,0),datetime.time(18,30),datetime.time(19,0),datetime.time(19,30),datetime.time(20,0),datetime.time(20,30)
]
unique_times_East = [ datetime.time(7, 0), datetime.time(7, 30), datetime.time(8, 0)
                ,datetime.time(8, 30),datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),
                 datetime.time(18, 0), datetime.time(18, 30), datetime.time(19, 0)
]
month_list=['2025-02-15']
fill_na=ETL(month_list,data,holiday_data,unique_times_East)
fill_na_predict=ETL_(month_list,data,holiday_data,unique_times_East)

X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
            [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data_1','years_data_2'] + [f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                add_columns
            
X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
            [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data_1','years_data_2'] + [f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                add_columns
date_cols=[f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()]
cal_cols=[f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()]
cols_offered=[col for col in now_lstm_data.columns if col not in date_cols]
lstm_data = fill_na.dropna(axis = 0)
lstm_data
for i in X_columns:
    try:
        fill_na_predict[i]
    except:
        fill_na_predict[i] = 0
lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])
for i in lstm_data.columns:
    try:
        fill_na_predict[i]
    except:
        fill_na_predict[i] = 0
#all_predict,all_feature=Train_Model(lstm_data,fill_na_predict,unique_times_East)


In [171]:
combined_data

,offered,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered
2021-12-21,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-22,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [170]:
fill_na

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data_1,years_data_2,year,month,day,hour,minute,datetime,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2023,year_2024,year_2025,week_of_year,weekday,week_of_year_1,week_of_year_2,week_of_year_3,week_of_year_4,week_of_year_5,week_of_year_6,week_of_year_7,week_of_year_8,week_of_year_9,week_of_year_10,week_of_year_11,week_of_year_12,week_of_year_13,week_of_year_14,week_of_year_15,week_of_year_16,week_of_year_17,week_of_year_18,week_of_year_19,week_of_year_20,week_of_year_21,week_of_year_22,week_of_year_23,week_of_year_24,week_of_year_25,week_of_year_26,week_of_year_27,week_of_year_28,week_of_year_29,week_of_year_30,week_of_year_31,week_of_year_32,week_of_year_33,week_of_year_34,week_of_year_35,week_of_year_36,week_of_year_37,week_of_year_38,week_of_year_39,week_of_year_40,week_of_year_41,week_of_year_42,week_of_year_43,week_of_year_44,week_of_year_45,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,6.0,12.0,0.0,12669.636364,344.121212,10.0,0.0,10069.166667,169.047619,5.0,0.0,7269.000000,104.000000,4.0,0.0,3080.000000,68.000000,2